# Home Sales Analysis - Challenge 22
*Student: Megan Neal*
*Instructor: Brandon Knox*

This notebook analyzes home sales data using PySpark SQL functions.

In [ ]:
# Import necessary packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import round
import time

# Create a SparkSession
spark = SparkSession.builder.appName("HomeSalesAnalysis").getOrCreate()
print("SparkSession initialized successfully!")

In [ ]:
# Read the home sales data
home_sales_df = spark.read.option("header", "true").csv("home_sales_revised.csv")

# Create temporary table
home_sales_df.createOrReplaceTempView("home_sales")
print("Data loaded and temporary table created! Preview of the data:")
home_sales_df.show(5)

## Query 1: Average price of 4-bedroom houses by year

In [ ]:
# Find average price of 4-bedroom houses by year
query1 = '''
    SELECT 
        YEAR(date_built) as year_built,
        ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY YEAR(date_built)
    ORDER BY year_built
'''
result1 = spark.sql(query1)
print("Average price of 4-bedroom houses by year:")
result1.show()

## Query 2: Average price of homes with 3 beds and 3 baths by year

In [ ]:
# Average price by year for 3 bed, 3 bath homes
query2 = '''
    SELECT 
        YEAR(date_built) as year_built,
        ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE bedrooms = 3 
    AND bathrooms = 3
    GROUP BY YEAR(date_built)
    ORDER BY year_built
'''
result2 = spark.sql(query2)
print("Average price of 3-bed, 3-bath homes by year:")
result2.show()

## Query 3: Average price of homes with specific criteria by year

In [ ]:
# Average price for homes with specific criteria
query3 = '''
    SELECT 
        YEAR(date_built) as year_built,
        ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    WHERE bedrooms = 3 
    AND bathrooms = 3 
    AND floors = 2 
    AND sqft_living >= 2000
    GROUP BY YEAR(date_built)
    ORDER BY year_built
'''
result3 = spark.sql(query3)
print("Average price of homes meeting specific criteria by year:")
result3.show()

## Query 4: Average price by view rating for expensive homes

In [ ]:
# Measure runtime for uncached query
start_time = time.time()

query4 = '''
    SELECT 
        view,
        ROUND(AVG(price), 2) as avg_price
    FROM home_sales
    GROUP BY view
    HAVING AVG(price) >= 350000
    ORDER BY view
'''
result4 = spark.sql(query4)
print("Average price by view rating (uncached):")
result4.show()

end_time = time.time()
print("Runtime without caching: ", round(end_time - start_time, 2), "seconds")

## Cache the temporary table

In [ ]:
# Cache the temporary table
spark.catalog.cacheTable("home_sales")

# Verify the table is cached
is_cached = spark.catalog.isCached("home_sales")
print("Is table cached?", is_cached)

# Run the query on cached data
start_time = time.time()
cached_result = spark.sql(query4)
print("
Average price by view rating (cached):")
cached_result.show()
end_time = time.time()
print("Runtime with caching: ", round(end_time - start_time, 2), "seconds")

## Create parquet table

In [ ]:
# Partition by date_built and save as parquet
home_sales_df.write.partitionBy("date_built").mode("overwrite").parquet("./home_sales_partitioned_parquet")

# Create temporary table from parquet data
parquet_df = spark.read.parquet("./home_sales_partitioned_parquet")
parquet_df.createOrReplaceTempView("home_sales_parquet")

# Run query on parquet table
start_time = time.time()
parquet_result = spark.sql(query4.replace("home_sales", "home_sales_parquet"))
print("Average price by view rating (parquet):")
parquet_result.show()
end_time = time.time()
print("Runtime with parquet: ", round(end_time - start_time, 2), "seconds")

## Uncache the temporary table

In [ ]:
# Uncache the temporary table
spark.catalog.uncacheTable("home_sales")

# Verify it is uncached
is_cached = spark.catalog.isCached("home_sales")
print("Is table still cached?", is_cached)